In [19]:
import numpy as np
import tensorflow as tf
from sklearn import linear_model
import utils
import time
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import scipy
plt.ioff()
import sys
from data_preprocess import get_data
import json
#tf.compat.v1.enable_eager_execution()

def sample_perturbation(data_point, regularizer = 20, learning_rate = 3e-2, num_steps = 200):
    """
    Calculates ratio between perturbed loss and original loss

    parameters: 
        data_point: tuple of x, y
            x: tensor of shape (d, )
            y: one-hot encoded tensor of shape (2, )
        regularizer (float): regularizer constant for fair metric
        learning_rate (float): step size for gradient ascend
        num_steps (int): number of steps in gradient ascend

    return:
        float; ratio of entropy losses for perturbed and original sample
    """
    x, y = data_point
    x = tf.reshape(x, (1, -1))
    y = tf.reshape(y, (1, -1))
    x_start = x
    for i in range(num_steps):
        with tf.GradientTape() as g:
            g.watch(x)
            prob = graph(x)
            perturb = utils.unprotected_direction(x-x_start, sensetive_directions)
            loss = utils.EntropyLoss(y, prob)  - regularizer  * tf.norm(perturb)**2

        gradient = g.gradient(loss, x)
        x = x + learning_rate * gradient/((i+1) ** (2/3))
        print(loss)

    return_loss = utils.EntropyLoss(y, graph(x)) / utils.EntropyLoss(y, graph(x_start))
    print('done')
    return return_loss.numpy()

def SimpleDense(variable):
    w, b = variable
    w = tf.cast(w, dtype = tf.float32)
    b = tf.cast(b, dtype = tf.float32)
    return lambda x: tf.matmul(x, w) + b


In [20]:
seed_data, seed_model = 7751, 43757
x_train, x_test, y_train, y_test, _, y_sex_test, y_race_test = get_data(seed_data)



sensetive_directions = []
protected_regression = linear_model.LogisticRegression(fit_intercept = True)
protected_regression.fit(x_test.numpy(), y_sex_test)
sensetive_directions.append(protected_regression.coef_.reshape((-1,)))
protected_regression.fit(x_test.numpy(), y_race_test)
sensetive_directions.append(protected_regression.coef_.reshape((-1,)))
sensetive_directions = np.array(sensetive_directions)

sensetive_directions = scipy.linalg.orth(sensetive_directions.T).T
for i, s in enumerate(sensetive_directions):
        #while np.linalg.norm(s) != 1:
    s = s/ np.linalg.norm(s)
    sensetive_directions[i] = s
sensetive_directions = tf.cast(sensetive_directions, dtype = tf.float32)


with open(f'models/data_{seed_data}_{seed_model}.txt', 'r') as f:
    weight = json.load(f)

weights = [np.array(w) for w in weight]

def graph(x):
    layer1 = SimpleDense([weights[0], weights[1]])
    layer2 = SimpleDense([weights[2], weights[3]])
    out = tf.nn.relu(layer1(x))
    out = layer2(out)
    prob = tf.nn.softmax(out)
    return prob

(5278, 8)
['sex', 'race', 'priors_count', 'age_cat=25 to 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'c_charge_degree=F', 'c_charge_degree=M']
sex
race


In [25]:
lr = 5e-2
data = x_test[0], y_test[0]
sample_perturbation(data, regularizer=50,\
             learning_rate=lr, num_steps=50)

tf.Tensor(0.5782235, shape=(), dtype=float32)
tf.Tensor(0.5778041, shape=(), dtype=float32)
tf.Tensor(0.5692901, shape=(), dtype=float32)
tf.Tensor(0.5582235, shape=(), dtype=float32)
tf.Tensor(0.56039625, shape=(), dtype=float32)
tf.Tensor(0.57307905, shape=(), dtype=float32)
tf.Tensor(0.58264685, shape=(), dtype=float32)
tf.Tensor(0.5858533, shape=(), dtype=float32)
tf.Tensor(0.5868089, shape=(), dtype=float32)
tf.Tensor(0.5873734, shape=(), dtype=float32)
tf.Tensor(0.5878782, shape=(), dtype=float32)
tf.Tensor(0.58835214, shape=(), dtype=float32)
tf.Tensor(0.5888, shape=(), dtype=float32)
tf.Tensor(0.5892248, shape=(), dtype=float32)
tf.Tensor(0.58962965, shape=(), dtype=float32)
tf.Tensor(0.59001666, shape=(), dtype=float32)
tf.Tensor(0.5903878, shape=(), dtype=float32)
tf.Tensor(0.59074444, shape=(), dtype=float32)
tf.Tensor(0.5910883, shape=(), dtype=float32)
tf.Tensor(0.59142, shape=(), dtype=float32)
tf.Tensor(0.5917409, shape=(), dtype=float32)
tf.Tensor(0.59205174, shape=(), 

1.0355872